---
jupyter: python3
---

In [1]:
import json
import time

import pandas as pd

from CryptoFraudDetection.elasticsearch.data_insertion import insert_dict
from CryptoFraudDetection.scraper.google_results import GoogleResultsScraper
from CryptoFraudDetection.utils.enums import LoggerMode, ScraperNotebookMode
from CryptoFraudDetection.utils.logger import Logger

logger_ = Logger(name="scrape_reddit_metadata", level=LoggerMode.DEBUG, log_dir="../logs")

In [2]:
MODE = ScraperNotebookMode.WRITE

In [3]:
# read data/raw/coins.json
with open("../data/raw/coins.json") as f:
    coins = json.load(f)
coins = sorted(coins, key=lambda coin: coin["max_market_cap_e9"], reverse=False)
coins[0]

{'name': 'Safe Moon',
 'symbol': 'SAFEMOON',
 'fraud': True,
 'test': True,
 'max_market_cap_e9': 0.0,
 'start_date': '2022-01-17',
 'subreddits': ['CryptoCurrency',
  'CryptoMoonShots',
  'CryptoMarkets',
  'Crypto',
  'SafeMoon'],
 'end_date': '2023-10-31'}

In [4]:
def get_next_proxy(
    link="https://api.proxyscrape.com/v4/free-proxy-list/get?request=display_proxies&proxy_format=protocolipport&format=csv&timeout=2000",
):
    proxy_list = pd.read_csv(link)
    proxy_list = proxy_list.sample(1)
    return proxy_list.iloc[0]

In [ ]:
proxy_info = get_next_proxy()
logger_.info(f"Using proxy {proxy_info.protocol}://{proxy_info.ip}:{proxy_info.port}")

N_SITES = 100
if MODE == ScraperNotebookMode.WRITE:
    for coin in coins:
        for subreddit in coin["subreddits"]:
            scrape_successful = False
            while not scrape_successful:
                try:
                    logger_.info(f"Scraping {coin['name']} in {subreddit}")
                    query = f"{coin['name']} site:reddit.com/r/{subreddit} {"before:" + coin['end_date'] if coin.get('end_date') else ""} {"after:" + coin['start_date'] }"
                    query = query.replace("  ", " ")
                    logger_.debug(f"Query: {query}")
                    scraper = GoogleResultsScraper(logger=logger_)
                    results = scraper.get_main_results(
                        query,
                        n_sites=N_SITES,
                        headless=True,
                        proxy_protocol=proxy_info.protocol,
                        proxy_address=f"{proxy_info.ip}:{proxy_info.port}",
                    )
                    insert_dict(
                        logger=logger_,
                        index="reddit_metadata_100",
                        data_dict=results,
                    )
                    scrape_successful = True
                    time.sleep(5)
                except Exception as e:
                    logger_.warning("Detected bot, proxy not working or other error")
                    proxy_info = get_next_proxy()
                    logger_.info(f"Using proxy {proxy_info.protocol}://{proxy_info.ip}:{proxy_info.port}")